In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Armado de features**

In [72]:
window = 5

In [73]:
if window <= 4:
    auctions_label = pd.read_pickle('data/auctions_w'+str(window)+'_label.pkl')
    installs_label = pd.read_pickle('data/installs_w'+str(window)+'_label.pkl')

In [74]:
target = pd.read_csv('data/target_competencia_ids.csv')
target['ref_hash'] = target['ref_hash'].apply(lambda x: int(str(x)[:-3]))
target.drop_duplicates('ref_hash', inplace=True)

In [75]:
time_windows = [('2019-04-18 00:00:00.000000', '2019-04-21 00:00:00.000000'), ('2019-04-19 00:00:00.000000', '2019-04-22 00:00:00.000000'), 
                ('2019-04-20 00:00:00.000000', '2019-04-23 00:00:00.000000'), ('2019-04-21 00:00:00.000000', '2019-04-24 00:00:00.000000'), 
                ('2019-04-24 00:00:00.000000', '2019-04-27 00:00:00.000000')]
time_labels = [('2019-04-21 00:00:00.000000', '2019-04-24 00:00:00.000000'), ('2019-04-22 00:00:00.000000', '2019-04-25 00:00:00.000000'), 
               ('2019-04-23 00:00:00.000000', '2019-04-26 00:00:00.000000'), ('2019-04-24 00:00:00.000000', '2019-04-27 00:00:00.000000')]
days = [18, 19, 20, 21, 24]

In [76]:
auctions = pd.read_pickle('data/auctions_w'+str(window)+'.pkl')
gb = auctions.groupby('ref_hash')
auctions['n_auctions'] = gb['date'].transform('count')
auctions['last_auction'] = gb['date'].transform('max')
auctions['first_auction'] = gb['date'].transform('min')
auctions['diff_auctions'] = (auctions['last_auction'] - auctions['first_auction']).dt.total_seconds()
auctions['mean_time_auction'] = 0
auctions.loc[auctions['n_auctions'] > 1, 'mean_time_auction'] = ((auctions['last_auction'] - auctions['first_auction'])/ \
                                 (auctions['n_auctions'] -1)).dt.total_seconds()
auctions['first_auction_sec'] = (auctions['first_auction'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
auctions['last_auction_sec'] = (auctions['last_auction'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
auctions['last_auction_sec_to_end'] = (pd.Timestamp(time_windows[window-1][1]) - auctions['last_auction']).dt.total_seconds()
auctions['ref_type_id_1'] = auctions['ref_type_id'].apply(lambda x: 1 if x==1 else 0)
auctions['ref_type_id_7'] = auctions['ref_type_id'].apply(lambda x: 1 if x==7 else 0)
auctions['day'] = (auctions['date'].dt.day) - days[window-1]
gb = auctions.groupby('ref_hash')
auctions = auctions.join(gb['source_id'].value_counts().unstack().add_prefix('source_'), on='ref_hash')
auctions = pd.get_dummies(auctions, columns=['day'])
gb = auctions.groupby('ref_hash')
auctions['day_0'] = gb['day_0'].transform('sum')
auctions['day_1'] = gb['day_1'].transform('sum')
auctions['day_2'] = gb['day_2'].transform('sum')

In [77]:
auctions.drop_duplicates(subset='ref_hash', inplace=True)
auctions.drop(columns=['date', 'ref_type_id', 'last_auction', 'first_auction', 'source_id'], inplace=True)
auctions = auctions.fillna(0)

In [78]:
clicks = pd.read_pickle('data/clicks_w'+str(window)+'.pkl')
clicks.drop(columns=['action_id', 'agent_device'])
gb = clicks.groupby('ref_hash')
clicks['n_clicks'] = gb['created'].transform('count')
clicks['last_click'] = gb['created'].transform('max')
clicks['first_click'] = gb['created'].transform('min')
clicks['diff_clicks'] = (clicks['last_click'] - clicks['first_click']).dt.total_seconds()
clicks['mean_time_click'] = 0
clicks.loc[clicks['n_clicks'] > 1, 'mean_time_click'] = ((clicks['last_click'] - clicks['first_click'])/ \
                                 (clicks['n_clicks'] -1)).dt.total_seconds()
clicks['first_click_sec'] = (clicks['first_click'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
clicks['last_click_sec'] = (clicks['last_click'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
clicks['wifi_connection'] = clicks['wifi_connection'].map({True: 1, False: 0})
clicks['timeToClick_mean'] = clicks.groupby('ref_hash')['timeToClick'].transform('mean')
clicks.loc[clicks.touchX == 'Infinity', 'touchX'] = 1
clicks.loc[clicks.touchY == 'Infinity', 'touchY'] = 10
clicks["touchX"] = pd.to_numeric(clicks["touchX"])
clicks["touchY"] = pd.to_numeric(clicks["touchY"])
clicks['touch_bottom'] = clicks['touchY'].apply(lambda x: 1 if x<=1 else 0)
clicks['touch_bottom2'] = clicks['touchY'].apply(lambda x: 1 if x>1 and x<=2 else 0)

top_10_carrier_id = clicks['carrier_id'].value_counts().head(10)
clicks.loc[(~clicks['carrier_id'].isin(top_10_carrier_id.index))&(clicks['carrier_id'].notnull()), 'carrier_id'] = 'Other'

gb = clicks.groupby('ref_hash')
clicks = clicks.join(gb['advertiser_id'].value_counts().unstack().add_prefix('advertiser_id_'), on='ref_hash')
clicks = clicks.join(gb['source_id'].value_counts().unstack().add_prefix('source_id_'), on='ref_hash')
clicks['touchX_mean'] = gb['touchX'].transform('mean')
clicks['touchY_mean'] = gb['touchY'].transform('mean')
clicks['touchs_in_bottom'] = gb['touch_bottom'].transform('sum')
clicks['touchs_in_bottom2'] = gb['touch_bottom2'].transform('sum')
clicks['latitude_mean'] = gb['latitude'].transform('mean')
clicks['longitude_mean'] = gb['longitude'].transform('mean')
clicks['timeToClick_mean'] = clicks['timeToClick_mean'].fillna(clicks['timeToClick_mean'].mean())
clicks['touchX_mean'] = clicks['touchX_mean'].fillna(clicks['touchX_mean'].mean())
clicks['touchY_mean'] = clicks['touchY_mean'].fillna(clicks['touchY_mean'].mean())

In [79]:
clicks.drop_duplicates(subset='ref_hash', inplace=True)
clicks = clicks.drop(columns=['advertiser_id', 'source_id', 'created', 'country_code', 'latitude', 'longitude', 'wifi_connection', 'carrier_id', 'trans_id', 'agent_device',
       'os_minor', 'os_major', 'specs_brand', 'brand', 'timeToClick', 'touchX', 'touchY', 'ref_type', 'last_click', 'first_click', 'touch_bottom', 'touch_bottom2', 'action_id'])

In [80]:
events = pd.read_pickle('data/events_w'+str(window)+'.pkl')
events.drop(columns=['event_uuid', 'ip_address', 'index', 'device_countrycode', 'trans_id'], inplace=True)

events['attributed'] = events['attributed'].map({True: 1, False: 0})
events['wifi'] = events['wifi'].map({True: 1, False: 0})
top_15_events_id = events['event_id'].value_counts().head(15)
events.loc[(~events['event_id'].isin(top_15_events_id.index))&(events['event_id'].notnull()), 'event_id'] = 'Other'
top_20_application_id = events['application_id'].value_counts().head(20)
events.loc[(~events['application_id'].isin(top_20_application_id.index))&(events['application_id'].notnull()), 'application_id'] = 'Other'
#top_15_device_os_version = events['device_os_version'].value_counts().head(15)
#events.loc[(~events['device_os_version'].isin(top_15_device_os_version.index))&(events['device_os_version'].notnull()), 'device_os_version'] = 'Other'
#top_15_device_brand = events['device_brand'].value_counts().head(15)
#events.loc[(~events['device_brand'].isin(top_15_device_brand.index))&(events['device_brand'].notnull()), 'device_brand'] = 'Other'
#top_15_device_model = events['device_model'].value_counts().head(15)
#events.loc[(~events['device_model'].isin(top_15_device_model.index))&(events['device_model'].notnull()), 'device_model'] = 'Other'
top_10_session_user_agent = events['session_user_agent'].value_counts().head(10)
events.loc[(~events['session_user_agent'].isin(top_10_session_user_agent.index))&(events['session_user_agent'].notnull()), 'session_user_agent'] = 'Other'
#top_10_carrier = events['carrier'].value_counts().head(10)
#events.loc[(~events['carrier'].isin(top_10_carrier.index))&(events['carrier'].notnull()), 'carrier'] = 'Other'
top_15_kind = events['kind'].value_counts().head(15)
events.loc[(~events['kind'].isin(top_15_kind.index))&(events['kind'].notnull()), 'kind'] = 'Other'
#top_15_device_language = events['device_language'].value_counts().head(15)
#events.loc[(~events['device_language'].isin(top_15_device_language.index))&(events['device_language'].notnull()), 'device_language'] = 'Other'

gb = events.groupby('ref_hash')
events = events.join(gb['event_id'].value_counts().unstack().add_prefix('event_id_'), on='ref_hash')
events = events.join(gb['application_id'].value_counts().unstack().add_prefix('application_id_'), on='ref_hash')
#events = events.join(gb['device_os_version'].value_counts().unstack().add_prefix('device_os_version_'), on='ref_hash')
#events = events.join(gb['device_brand'].value_counts().unstack().add_prefix('device_brand_'), on='ref_hash')
#events = events.join(gb['device_model'].value_counts().unstack().add_prefix('device_model_'), on='ref_hash')
#events = events.join(gb['carrier'].value_counts().unstack().add_prefix('carrier_'), on='ref_hash')
events = events.join(gb['session_user_agent'].value_counts().unstack().add_prefix('session_user_agent_'), on='ref_hash')
events = events.join(gb['kind'].value_counts().unstack().add_prefix('kind_'), on='ref_hash')
#events = events.join(gb['device_language'].value_counts().unstack().add_prefix('device_language_'), on='ref_hash')

events['n_events'] = gb['date'].transform('count')
events['attributed_events_mean'] = gb['attributed'].transform('mean')
events['wifi_events_mean'] = gb['wifi'].transform('mean')
events['first_event'] = gb['date'].transform('min')
events['last_event'] = gb['date'].transform('max')
events['diff_events'] = (events['last_event'] - events['first_event']).dt.total_seconds()
events['mean_time_events'] = 0
events.loc[events['n_events'] > 1, 'mean_time_events'] = ((events['last_event'] - events['first_event'])/ \
                                 (events['n_events'] -1)).dt.total_seconds()
events['first_event_sec'] = (events['first_event'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
events['last_event_sec'] = (events['last_event'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()

In [81]:
events.drop_duplicates(subset='ref_hash', inplace=True)
events = events.drop(columns=['date', 'event_id', 'ref_type', 'application_id', 'attributed', 'device_os_version', 'device_brand', 'device_model', 
       'device_city', 'session_user_agent', 'user_agent', 'carrier', 'kind', 'device_os', 'wifi', 'connection_type', 'device_language', 'first_event', 'last_event'])

In [82]:
installs = pd.read_pickle('data/installs_w'+str(window)+'.pkl')
installs = installs.drop(columns=['device_countrycode', 'ip_address', 'event_uuid', 'click_hash', 'device_brand', 'device_model'])
#installs = installs.drop(columns=['device_language', 'device_countrycode', 'ip_address', 'event_uuid', 'device_brand', 'device_model', 'click_hash', 'session_user_agent'])
installs.loc[installs['kind'] == 'OPEN', 'kind'] = 'Open'
installs.loc[installs['kind'] == 'app open', 'kind'] = 'app_open'
installs.loc[installs['kind'] == 'af app open', 'kind'] = 'af_app_opened'
installs.loc[installs['kind'] == 'af_app_opend', 'kind'] = 'af_app_opened'
installs.loc[installs['kind'] == 'Session Begin', 'kind'] = 'sessionbegin'
installs.loc[installs['kind'] == 'signed in', 'kind'] = 'Sign In'
top_20_installs_kind = installs['kind'].value_counts().head(20)
installs.loc[(~installs['kind'].isin(top_20_installs_kind.index))&(installs['kind'].notnull()), 'kind'] = 'Other'
top_15_installs_app = installs['application_id'].value_counts().head(15)
installs.loc[(~installs['application_id'].isin(top_15_installs_app.index))&(installs['application_id'].notnull()), 'application_id'] = 'Other'
installs['created'] = pd.to_datetime(installs['created'], format='%Y-%m-%d %H:%M:%S')
installs['wifi_installs'] = installs['wifi'].map({True: 1, False: 0})
installs['attributed'] = installs['attributed'].map({True: 1, False: 0})
installs['implicit'] = installs['implicit'].map({True: 1, False: 0})
gb = installs.groupby('ref_hash')
installs = installs.join(gb['kind'].value_counts().unstack().add_prefix('kind_'), on='ref_hash')
installs = installs.join(gb['application_id'].value_counts().unstack().add_prefix('application_id_'), on='ref_hash')
installs['wifi_installs_mean'] = gb['wifi_installs'].transform('mean')
installs['attributed_installs_mean'] = gb['attributed'].transform('mean')
installs['n_installs'] = gb['created'].transform('count')
installs['last_install'] = gb['created'].transform('max')
installs['first_install'] = gb['created'].transform('min')
installs['diff_installs'] = (installs['last_install'] - installs['first_install']).dt.total_seconds()
installs['mean_time_install'] = 0
installs.loc[installs['n_installs'] > 1, 'mean_time_install'] = ((installs['last_install'] - installs['first_install'])/ \
                                 (installs['n_installs'] -1)).dt.total_seconds()
installs['first_install_sec'] = (installs['first_install'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
installs['last_install_sec'] = (installs['last_install'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()

In [83]:
installs.drop(columns=['created', 'application_id', 'ref_type', 'attributed',
       'implicit', 'user_agent', 'kind', 'wifi', 'trans_id', 'device_language',
       'wifi_installs', 'last_install', 'first_install', 'session_user_agent'], inplace=True)
installs.drop_duplicates(subset='ref_hash', inplace=True)

In [84]:
model = auctions

In [85]:
model = pd.merge(model, installs, how='left', on='ref_hash')
model = pd.merge(model, events, how='left', on='ref_hash')
model = pd.merge(model, clicks, how='left', on='ref_hash')

In [86]:
if window <= 4:
    model = pd.merge(model, auctions_label, on='ref_hash', how='left')
    model = pd.merge(model, installs_label, on='ref_hash', how='left')
    model['time_appearence'] = model['time_appearence'].fillna(259200)
    model['time_appearence_install'] = model['time_appearence_install'].fillna(259200)
model.drop_duplicates('ref_hash', inplace=True)
uses = 'all'
model.to_pickle('data/model/model_' +uses+'_w'+str(window)+'.pkl')